###  Uso de combinaciones externas

Aunque no es tan común como las combinaciones internas, el uso de combinaciones externas en una consulta de varias tablas puede proporcionar una vista alternativa de los datos empresariales. Como sucede con las combinaciones internas, expresará una relación lógica entre las tablas. Pero no solo recuperará las filas con atributos coincidentes, sino también todas las filas presentes en una tablas o las dos, independientemente de si hay o no una coincidencia en la otra tabla.

Anteriormente, ha aprendido a usar INNER JOIN para buscar filas coincidentes entre dos tablas. Como ha visto, el procesador de consultas genera los resultados de una consulta INNER JOIN filtrando las filas que no cumplen las condiciones expresadas en el predicado de la cláusula ON. El resultado es que solo se devuelven las filas con una fila coincidente en la otra tabla. Con OUTER JOIN, puede optar por mostrar todas las filas que tienen filas coincidentes entre las tablas, además de todas las filas que no tienen ninguna coincidencia en la otra tabla. Ahora se verá un ejemplo y, después, se examinará el proceso.

En primer lugar, examine la consulta siguiente, escrita con INNER JOIN:

In [ ]:
SELECT emp.FirstName, ord.Amount
FROM HR.Employee AS emp
INNER JOIN Sales.SalesOrder AS ord
    ON emp.EmployeeID = ord.EmployeeID;

Ahora, se examinará la siguiente consulta, escrita como LEFT OUTER JOIN:

In [ ]:
SELECT emp.FirstName, ord.Amount
FROM HR.Employee AS emp
LEFT OUTER JOIN Sales.SalesOrder AS ord
    ON emp.EmployeeID = ord.EmployeeID;

En este ejemplo se usa un operador LEFT OUTER JOIN, que indica al procesador de consultas que conserve todas las filas de la tabla de la izquierda (HR.Employee) y muestre los valores Amount para las filas coincidentes en Sales.SalesOrder. Pero se devuelven todos los empleados, independientemente de si han realizado o no un pedido de ventas. En lugar del valor Amount, la consulta devolverá NULL para los empleados sin pedidos de ventas correspondientes.

### Sintaxis de OUTER JOIN
Las combinaciones externas se expresan mediante las palabras clave LEFT, RIGHT o FULL, que se colocan por delante de OUTER JOIN. El propósito de la palabra clave es indicar qué tabla (en qué lado de la palabra clave JOIN) se debe conservar y mostrar todas sus filas, haya coincidencias o no.

Al usar LEFT, RIGHT o FULL para definir una combinación, puede omitir la palabra clave OUTER como se muestra aquí:

In [ ]:
SELECT emp.FirstName, ord.Amount
FROM HR.Employee AS emp
LEFT JOIN Sales.SalesOrder AS ord
    ON emp.EmployeeID = ord.EmployeeID;

Pero como sucede con la palabra clave INNER, a menudo resulta útil escribir código que indique de forma explícita el tipo de combinación que se usa.

Al escribir consultas mediante OUTER JOIN, tenga en cuenta las instrucciones siguientes:

- Como ha visto, se prefieren alias de tabla, no solo para la lista SELECT, sino también para la cláusula ON.
- Como sucede con INNER JOIN, se puede realizar una operación OUTER JOIN en una sola columna coincidente o en varios atributos coincidentes.
- A diferencia de INNER JOIN, el orden en el que las tablas se enumeran y se unen en la cláusula FROM sí importa con OUTER JOIN, ya que determinará si elige LEFT o RIGHT para la combinación.
- Las combinaciones de varias tablas son más complejas cuando se usa OUTER JOIN. La presencia de valores NULL en los resultados de OUTER JOIN puede provocar problemas si los resultados intermedios se combinan a una tercera tabla. El predicado de la segunda combinación puede filtrar las filas con valores NULL.
- Para mostrar solo las filas en las que no existe ninguna coincidencia, agregue una prueba de NULL en una cláusula WHERE después de un predicado OUTER JOIN.
FULL OUTER JOIN rara vez se usa. Devuelve todas las filas coincidentes entre las dos tablas, más todas las filas de la primera tabla sin coincidencia en la segunda, además de todas las filas de la segunda sin coincidencia en la primera.
- No hay ninguna manera de predecir el orden en que se devolverán las filas sin una cláusula ORDER BY. No hay ninguna manera de saber si primero se devolverán las filas coincidentes o las no coincidentes.

### Uso de combinaciones cruzadas
Una combinación cruzada es simplemente un producto cartesiano de las dos tablas. Con la sintaxis ANSI SQL-89, puede excluir el filtro que conecta las dos tablas para crear una combinación cruzada. Con la sintaxis ANSI-92, es un poco más difícil, lo que es positivo porque, en general, una combinación cruzada no es algo que normalmente le interesará. Con la sintaxis ANSI-92, es muy poco probable que termine con una combinación cruzada accidentalmente.

Para crear de forma explícita un producto cartesiano, use el operador CROSS JOIN.

Esta operación crea un conjunto de resultados con todas las combinaciones posibles de las filas de entrada:

In [ ]:
SELECT <select_list>
FROM table1 AS t1
CROSS JOIN table2 AS t2;

Aunque este resultado no suele ser una salida deseada, hay algunas aplicaciones prácticas para escribir una operación CROSS JOIN explícita:

Crear una tabla de números, con una fila para cada valor posible de un intervalo.
Generar grandes volúmenes de datos para pruebas. Cuando se le aplica una combinación cruzada a sí misma, una tabla con tan solo 100 filas puede generar fácilmente 10 000 filas de salida.
### Sintaxis de CROSS JOIN
Al escribir consultas mediante CROSS JOIN, tenga en cuenta las instrucciones siguientes:

No se realiza ninguna coincidencia de filas, por lo que no se usa ninguna cláusula ON. (Es un error usar una cláusula ON con CROSS JOIN).
Para usar la sintaxis ANSI SQL-92, separe los nombres de tabla de entrada con el operador CROSS JOIN.
La consulta siguiente es un ejemplo del uso de CROSS JOIN para crear todas las combinaciones de empleados y productos:

In [ ]:
SELECT emp.FirstName, prd.Name
FROM HR.Employee AS emp
CROSS JOIN Production.Product AS prd;

### Uso de autocombinaciones

Hasta ahora, en las combinaciones que se han usado participaban tablas diferentes. Es posible que haya escenarios en los que tenga que recuperar y comparar filas de una tabla con otras filas de la misma tabla. Por ejemplo, en una aplicación de recursos humanos, una tabla Employee podría incluir información sobre el jefe de cada empleado y almacenar el identificador del jefe en la propia fila del empleado. Cada jefe también aparece como empleado.

Para recuperar la información de los empleados y hacerla coincidir con el jefe relacionado, puede usar la tabla dos veces en la consulta, y combinarla con sí misma para los fines de la consulta.

In [ ]:
SELECT emp.FirstName AS Employee, 
       mgr.FirstName AS Manager
FROM HR.Employee AS emp
LEFT OUTER JOIN HR.Employee AS mgr
  ON emp.ManagerID = mgr.EmployeeID;

Los resultados de esta consulta incluyen una fila para cada empleado con el nombre de su jefe. El director general de la empresa no tiene ningún jefe. Para incluir al director general en los resultados, se usa una combinación externa y el nombre del jefe se devuelve como NULL para las filas en las que el campo ManagerID no tiene ningún campo EmployeeID coincidente.

Hay otros escenarios en los que querrá comparar filas de una tabla con otras filas de la misma tabla. Como ha visto, es bastante fácil comparar columnas de la misma fila mediante T-SQL, pero el método para comparar valores de filas diferentes (como una fila que almacena una hora de inicio y otra de la misma tabla que almacena una hora de finalización correspondiente) es menos obvio. Las autocombinaciones son una técnica útil para estos tipos de consultas.

Para realizar tareas como esta, debe tener en cuenta las instrucciones siguientes:

- Defina dos instancias de la misma tabla en la cláusula FROM y combínelas según sea necesario, mediante combinaciones internas o externas.
- Use alias de tabla para diferenciar las dos instancias de la misma tabla.
- Use la cláusula ON para proporcionar un filtro que compare las columnas de una instancia de la tabla con las columnas de la otra instancia.

### Comprender las subconsultas
Una subconsulta es una instrucción SELECT anidada dentro de otra consulta. La posibilidad de anidar una consulta dentro de otra mejorará su capacidad de crear consultas eficaces en T-SQL. En general, las subconsultas se evalúan una vez y proporcionan sus resultados a la consulta externa.

Trabajar con subconsultas
Una subconsulta es una instrucción SELECT anidada o incrustada en otra consulta. La consulta anidada, que es la subconsulta, se conoce como consulta interna. La consulta que contiene la consulta anidada es la consulta externa.

El propósito de una subconsulta es devolver resultados a la consulta externa. La forma de los resultados determinará si la subconsulta es una subconsulta escalar o multivalor:

- Las subconsultas escalares devuelven un solo valor. Las consultas externas deben procesar un único resultado.
- Las subconsultas multivalor devuelven un resultado muy similar a una tabla de una sola columna. Las consultas externas deben poder procesar varios valores.
Además de la elección entre subconsultas escalares y multivalor, las subconsultas pueden ser independientes o pueden correlacionarse con la consulta externa:

- Las subconsultas independientes se pueden escribir como consultas independientes, sin dependencias de la consulta externa. Una subconsulta independiente se procesa una vez, cuando la consulta externa se ejecuta y pasa sus resultados a esa consulta externa.
- Las subconsultas correlacionadas hacen referencia a una o varias columnas de la consulta externa y, por tanto, dependen de ella. Las subconsultas correlacionadas no se pueden ejecutar por separado desde la consulta externa.

### Usar subconsultas escalares o multivalor

Una subconsulta escalar es una instrucción SELECT interna dentro de una consulta externa, escrita para devolver un solo valor. Las subconsultas escalares se pueden usar en cualquier lugar de una instrucción T-SQL externa en la que se permita una expresión de un solo valor, como en una cláusula SELECT, una cláusula WHERE, una cláusula HAVING o incluso una cláusula FROM. También se pueden usar en instrucciones de modificación de datos, como UPDATE o DELETE.

Las subconsultas multivalor, como sugiere el nombre, pueden devolver más de una fila. Sin embargo, todavía devuelven una sola columna.

Subconsultas escalares
Imaginemos que desea recuperar los detalles del último pedido que se ha realizado, suponiendo que es el que tiene el valor SalesOrderID más alto.

Para buscar el valor SalesOrderID más alto, puede usar la consulta siguiente:

In [ ]:
SELECT MAX(SalesOrderID)
FROM Sales.SalesOrderHeader

Esta consulta devuelve un valor único que indica el valor más alto de OrderID en la tabla SalesOrderHeader.

Para obtener los detalles de este pedido, es posible que tenga que filtrar la tabla SalesOrderDetails en función del valor devuelto por la consulta anterior. Puede realizar esta tarea anidando la consulta para recuperar el valor máximo de SalesOrderID dentro de la cláusula WHERE de una consulta que recupera los detalles del pedido.

In [ ]:
SELECT SalesOrderID, ProductID, OrderQty
FROM Sales.SalesOrderDetail
WHERE SalesOrderID = 
   (SELECT MAX(SalesOrderID)
    FROM Sales.SalesOrderHeader);

Para escribir una subconsulta escalar, tenga en cuenta las siguientes directrices:

- Para indicar una consulta como subconsulta, escríbala entre paréntesis.
- Se admiten varios niveles de subconsultas en Transact-SQL. En este módulo, solo se considerarán las consultas de dos niveles (una consulta interna dentro de una consulta externa), pero se admiten hasta 32 niveles.
- Si la subconsulta no devuelve filas (un conjunto vacío), el resultado de la subconsulta es NULL. Si es posible en su escenario que no se devuelva ninguna fila, debe asegurarse de que la consulta externa puede controlar correctamente un valor NULL, además de otros resultados esperados.
- Por lo general, la consulta interna debe devolver una sola columna. La selección de varias columnas en una subconsulta casi siempre es un error. La única excepción es si la subconsulta se indica con la palabra clave EXISTS.


Una subconsulta escalar se puede usar en cualquier lugar de una consulta donde se espera un valor, incluida la lista SELECT. Por ejemplo, podríamos ampliar la consulta que recuperó los detalles del pedido más reciente para incluir la cantidad media de elementos que se pide, de modo que podamos comparar la cantidad pedida en el pedido más reciente con la media de todos los pedidos.

In [ ]:
SELECT SalesOrderID, ProductID, OrderQty,
    (SELECT AVG(OrderQty)
     FROM SalesLT.SalesOrderDetail) AS AvgQty
FROM SalesLT.SalesOrderDetail
WHERE SalesOrderID = 
    (SELECT MAX(SalesOrderID)
     FROM SalesLT.SalesOrderHeader);

### Subconsultas multivalor
Una subconsulta multivalor es adecuada para devolver resultados mediante el operador IN. En el ejemplo hipotético siguiente se devuelven los valores CustomerID y SalesOrderID de todos los pedidos realizados por los clientes de Canadá.

In [ ]:
SELECT CustomerID, SalesOrderID
FROM Sales.SalesOrderHeader
WHERE CustomerID IN (
    SELECT CustomerID
    FROM Sales.Customer
    WHERE CountryRegion = 'Canada');

En este ejemplo, si ejecutara solo la consulta interna, se devolvería una columna de valores CustomerID, con una fila para cada cliente de Canadá.

En muchos casos, las subconsultas multivalor se pueden escribir fácilmente mediante combinaciones. Por ejemplo, esta es una consulta que usa una combinación para devolver los mismos resultados del ejemplo anterior: 

In [ ]:
SELECT c.CustomerID, o.SalesOrderID
FROM Sales.Customer AS c
JOIN Sales.SalesOrderHeader AS o
    ON c.CustomerID = o.CustomerID
WHERE c. CountryRegion = 'Canada';

### Usar subconsultas independientes o correlacionadas

Anteriormente, examinamos las subconsultas independientes en las que la consulta interna es independiente de la consulta externa, se ejecuta una vez y devuelve sus resultados a la consulta externa. T-SQL también admite las subconsultas correlacionadas, en las que la consulta interna hace referencia a la columna de la consulta externa y conceptualmente se ejecuta una vez por fila.

### Trabajar con subconsultas correlacionadas
Al igual que las subconsultas independientes, las subconsultas correlacionadas son instrucciones SELECT anidadas dentro de una consulta externa. Las subconsultas correlacionadas también pueden ser subconsultas escalares o multivalor. Normalmente se usan cuando la consulta interna necesita hacer referencia a un valor en la consulta externa.

Sin embargo, a diferencia de las subconsultas independientes, hay algunas consideraciones especiales cuando se usan subconsultas correlacionadas:

- Las subconsultas correlacionadas no se pueden ejecutar por separado desde la consulta externa. Esta restricción complica las pruebas y la depuración.
- A diferencia de las subconsultas independientes, que se procesan una vez, las subconsultas correlacionadas se ejecutarán varias veces. Lógicamente, la consulta externa se ejecuta primero y, para cada fila devuelta, se procesa la consulta interna.

En el ejemplo siguiente se usa una subconsulta correlacionada para devolver el pedido más reciente de cada cliente. La subconsulta hace referencia a la consulta externa y hace referencia a su valor CustomerID en su cláusula WHERE. Para cada fila de la consulta externa, la subconsulta busca el identificador de pedido máximo del cliente al que se hace referencia en esa fila y la consulta externa comprueba si la fila que se está analizando es la fila con ese identificador de pedido.

In [ ]:
SELECT SalesOrderID, CustomerID, OrderDate
FROM SalesLT.SalesOrderHeader AS o1
WHERE SalesOrderID =
    (SELECT MAX(SalesOrderID)
     FROM SalesLT.SalesOrderHeader AS o2
     WHERE o2.CustomerID = o1.CustomerID)
ORDER BY CustomerID, OrderDate;

### Escritura de subconsultas correlacionadas
Para escribir subconsultas correlacionadas, tenga en cuenta las siguientes directrices:

- Escriba la consulta externa para aceptar el resultado devuelto adecuado de la consulta interna. Si la consulta interna es escalar, puede usar operadores de igualdad y comparación, como =, <, > y <>, en la cláusula WHERE. Si la consulta interna puede devolver varios valores, use un predicado IN. Cree un plan para controlar los resultados NULL.
- Identifique la columna de la consulta externa a la que hará referencia la subconsulta correlacionada. Declare un alias para la tabla que es el origen de la columna en la consulta externa.
- Identifique la columna de la tabla interna que se comparará con la columna de la tabla externa. Cree un alias para la tabla de origen, como hizo para la consulta externa.
- Escriba la consulta interna para recuperar valores de su origen, en función del valor de entrada de la consulta externa. Por ejemplo, use la columna externa en la cláusula WHERE de la consulta interna.

La correlación entre las consultas interna y externa se produce cuando la consulta interna hace referencia al valor externo para su comparación. Es esta correlación la que proporciona a la subconsulta su nombre.

### Trabajar con EXISTS
Además de recuperar valores de una subconsulta, T-SQL proporciona un mecanismo para comprobar si se devolverían resultados de una consulta. El predicado EXISTS determina si existen filas que cumplan una condición especificada, pero en lugar de devolverlas, devuelve TRUE o FALSE. Esta técnica es útil para validar los datos sin incurrir en la sobrecarga de recuperar y procesar los resultados.

Cuando una subconsulta está relacionada con la consulta externa mediante el predicado EXISTS, SQL Server controla los resultados de la subconsulta de una manera especial. En lugar de recuperar un valor escalar o una lista multivalor de la subconsulta, EXISTS simplemente comprueba si hay filas en el resultado.

Conceptualmente, un predicado EXISTS es equivalente a la recuperación de los resultados, el recuento de las filas devueltas y la comparación del recuento con cero. Compare las consultas siguientes, que devolverán detalles sobre los clientes que han realizado pedidos:

La primera consulta de ejemplo usa COUNT en una subconsulta:

In [ ]:
SELECT CustomerID, CompanyName, EmailAddress 
FROM Sales.Customer AS c 
WHERE
(SELECT COUNT(*) 
  FROM Sales.SalesOrderHeader AS o
  WHERE o.CustomerID = c.CustomerID) > 0;

La segunda consulta, que devuelve los mismos resultados, usa EXISTS:

In [ ]:
SELECT CustomerID, CompanyName, EmailAddress 
FROM Sales.Customer AS c 
WHERE EXISTS
(SELECT * 
  FROM Sales.SalesOrderHeader AS o
  WHERE o.CustomerID = c.CustomerID);

En el primer ejemplo, la subconsulta debe contar todas las repeticiones de cada custid que se encuentre en la tabla Sales.SalesOrderHeader y comparar los resultados del recuento con cero, simplemente para indicar que el cliente ha realizado pedidos.

En la segunda consulta, EXISTS devolverá TRUE para un custid en cuanto se haya encontrado un pedido pertinente en la tabla Sales.SalesOrderHeader. No es necesario realizar una contabilidad de cuentas completa de cada repetición. Tenga en cuenta también que, con el formulario EXISTS, la subconsulta no está restringida a la devolución de una sola columna. Aquí, aparece SELECT *. Las columnas devueltas son irrelevantes porque solo estamos comprobando si se devuelve alguna fila, no qué valores hay en esas filas.

Desde la perspectiva del procesamiento lógico, los dos formularios de consulta son equivalentes. Desde una perspectiva de rendimiento, el motor de base de datos puede tratar las consultas de forma diferente a medida que las optimiza para su ejecución. Considere la posibilidad de probar cada una de ellos para su propio uso.

 Nota

Si va a convertir una subconsulta mediante COUNT(*) en una con EXISTS, asegúrese de que la subconsulta usa SELECT * y no SELECT COUNT(*). SELECT COUNT(*) siempre devuelve una fila, de modo que EXISTS siempre devolverá TRUE.

Otra aplicación útil de EXISTS es la negación de la subconsulta con NOT, como en el ejemplo siguiente, que devolverá cualquier cliente que nunca haya realizado un pedido:

In [ ]:
SELECT CustomerID, CompanyName, EmailAddress 
FROM SalesLT.Customer AS c 
WHERE NOT EXISTS
  (SELECT * 
   FROM SalesLT.SalesOrderHeader AS o
   WHERE o.CustomerID = c.CustomerID);

SQL Server tendrá que devolver datos sobre los pedidos relacionados para los clientes que hayan realizado pedidos. Si se encuentra un custid en la tabla Sales.SalesOrderHeader, NOT EXISTS se evalúa como FALSE y la evaluación se completa rápidamente.

Para escribir consultas que usan EXISTS con subconsultas, tenga en cuenta las siguientes directrices:

- La palabra clave EXISTS sigue directamente a WHERE. Ningún nombre de columna (u otra expresión) la precede, a menos que también se utilice NOT.
- En la subconsulta, use SELECT *. La subconsulta no devuelve ninguna fila, por lo que no es necesario especificar ninguna columna.

## Uso de funciones integradas y GROUP BY en Transact-SQL 

#### Categorizar funciones integradas

Transact-SQL incluye muchas funciones integradas, desde funciones que convierten tipos de datos hasta funciones que agregan y analizan grupos de filas.

Las funciones de T-SQL se pueden clasificar de la siguiente manera:



- Escalar: Opera sobre un solo valor y devuelve otro valor.

- Lógicos: Compara varios valores para determinar una única salida.

- Clasificación: Opera sobre una partición (conjunto) de filas.

- Conjunto de filas: Devuelve una tabla virtual que se puede usar en una cláusula FROM de una instrucción T-SQL.

- Agregado: Toma uno o varios valores de entrada y devuelve un único valor de resumen.

#### Usar funciones escalares

Las funciones escalares devuelven un valor único y normalmente funcionan en una sola fila de datos. El número de valores de entrada que toman puede ser cero (por ejemplo, GETDATE), uno (por ejemplo, UPPER) o varios (por ejemplo, ROUND). Dado que las funciones escalares siempre devuelven un solo valor, se pueden usar en cualquier lugar en el que se necesite un único valor (el resultado). Se usan normalmente en cláusulas SELECT y predicados de cláusula WHERE. También se pueden usar en la cláusula SET de una instrucción UPDATE.

Las funciones escalares integradas se pueden organizar en muchas categorías, como cadena, conversión, lógica, matemática y otras. En este módulo se explican algunas funciones escalares comunes.

Estas son algunas de las consideraciones que hay que tener en cuenta al usar funciones escalares:

- Determinismo: si la función devuelve el mismo valor para el mismo estado de entrada y base de datos cada vez que se llama, se dice que es determinista. Por ejemplo, ROUND(1.1, 0) siempre devuelve el valor 1.0. Muchas funciones integradas son no deterministas. Por ejemplo, GETDATE() devuelve la fecha y hora actuales. Los resultados de las funciones no deterministas no se pueden indexar, lo que afecta a la capacidad del procesador de consultas de idear un buen plan para ejecutar la consulta.
- Intercalación: cuando se usan funciones que manipulan datos de caracteres, ¿qué intercalación se usará? Algunas funciones usan la intercalación (criterio de ordenación) del valor de entrada; otros usan la intercalación de la base de datos si no se proporciona ninguna intercalación de entrada.
#### Ejemplos de funciones escalares
En el momento de redactar la documentación de SQL Server, se enumeran más de 200 funciones escalares que abarcan varias categorías, entre las que se incluyen las siguientes:

Funciones de configuración
Funciones de conversión
Funciones de cursores
Funciones de fecha y hora
Funciones matemáticas
Funciones de metadatos
Funciones de seguridad
Funciones de cadena
Funciones del sistema
Funciones estadísticas del sistema
Funciones de texto y de imagen
No hay tiempo suficiente en este curso para describir cada función, pero en los ejemplos siguientes se muestran algunas funciones que se usan con frecuencia.

En el ejemplo hipotético siguiente se usan varias funciones de fecha y hora:



In [ ]:
SELECT  SalesOrderID,
    OrderDate,
        YEAR(OrderDate) AS OrderYear,
        DATENAME(mm, OrderDate) AS OrderMonth,
        DAY(OrderDate) AS OrderDay,
        DATENAME(dw, OrderDate) AS OrderWeekDay,
        DATEDIFF(yy,OrderDate, GETDATE()) AS YearsSinceOrder
FROM Sales.SalesOrderHeader;

In [ ]:
--En el ejemplo siguiente se incluyen algunas funciones matemáticas:
SELECT TaxAmt,
       ROUND(TaxAmt, 0) AS Rounded,
       FLOOR(TaxAmt) AS Floor,
       CEILING(TaxAmt) AS Ceiling,
       SQUARE(TaxAmt) AS Squared,
       SQRT(TaxAmt) AS Root,
       LOG(TaxAmt) AS Log,
       TaxAmt * RAND() AS Randomized
FROM Sales.SalesOrderHeader;

In [ ]:
-- En el ejemplo siguiente se usan algunas funciones de cadena:
SELECT  CompanyName,
        UPPER(CompanyName) AS UpperCase,
        LOWER(CompanyName) AS LowerCase,
        LEN(CompanyName) AS Length,
        REVERSE(CompanyName) AS Reversed,
        CHARINDEX(' ', CompanyName) AS FirstSpace,
        LEFT(CompanyName, CHARINDEX(' ', CompanyName)) AS FirstWord,
        SUBSTRING(CompanyName, CHARINDEX(' ', CompanyName) + 1, LEN(CompanyName)) AS RestOfName
FROM Sales.Customer;

### Funciones lógicas
Otra categoría de funciones permite determinar cuál de varios valores se va a devolver. Las funciones lógicas evalúan una expresión de entrada y devuelven un valor adecuado en función del resultado.

IIF
La función IIF evalúa una expresión de entrada booleana y devuelve un valor especificado si la expresión se evalúa como True, y un valor alternativo si la expresión se evalúa como False.

Por ejemplo, observe la siguiente consulta, que evalúa el tipo de dirección de un cliente. Si el valor es "Oficina principal", la expresión devuelve "Facturación". Para todos los demás valores de tipo de dirección, la expresión devuelve "Correo".




In [ ]:
SELECT AddressType,
      IIF(AddressType = 'Main Office', 'Billing', 'Mailing') AS UseAddressFor
FROM Sales.CustomerAddress;

#### CHOOSE
La función CHOOSE evalúa una expresión de entero y devuelve el valor correspondiente de una lista en función de su posición ordinal (basada en 1).

In [ ]:
SELECT SalesOrderID, Status,
CHOOSE(Status, 'Ordered', 'Shipped', 'Delivered') AS OrderStatus
FROM Sales.SalesOrderHeader;

--Si el estatus es Ordered pondrá 1, si es Shipped pondrá 2 so on ...

#### Utilizar funciones de clasificación y conjunto de filas

Las funciones de clasificación y conjunto de filas no son funciones escalares porque no devuelven un solo valor. Estas funciones aceptan un conjunto de filas como entrada y devuelven un conjunto de filas como salida.

#### Funciones de categoría
Las funciones de clasificación permiten realizar cálculos en un conjunto de filas definido por el usuario. Estas funciones incluyen funciones de clasificación, desplazamiento, agregado y distribución.

En este ejemplo se usa la función RANK para calcular una clasificación basada en ListPrice, con el precio más alto clasificado en 1:

In [ ]:
SELECT TOP 100 ProductID, Name, ListPrice,
RANK() OVER(ORDER BY ListPrice DESC) AS RankByPrice
FROM Production.Product AS p
ORDER BY RankByPrice;

/*La función RANK() crea un ranking pero no continuo, mas bien basado
en la posción en la tabla, esto es si hay 4 registros que tengan el mismo 
valor(el más alto) el sigueinte elemento no será el 2 si no el 5*/ 

#### OVER
Puede usar la cláusula OVER para definir particiones o agrupaciones dentro de los datos. Por ejemplo, la consulta siguiente amplía el ejemplo anterior para calcular las clasificaciones basadas en precios de los productos dentro de cada categoría.

In [ ]:
SELECT c.Name AS Category, p.Name AS Product, ListPrice,
  RANK() OVER(PARTITION BY c.Name ORDER BY ListPrice DESC) AS RankByPrice
FROM Production.Product AS p
JOIN Production.ProductCategory AS c
ON p.ProductCategoryID = c.ProductcategoryID
ORDER BY Category, RankByPrice;
/*Aquí se crea un rank() del mismo modo que el anterior pero
al mismo tiemp ose crea un rank por subcatetgoria */

### Uso de funciones de agregado

T-SQL proporciona funciones de agregado como SUM, MAX y AVG para realizar cálculos que toman varios valores y devuelven un único resultado.

Uso de funciones de agregado
La mayoría de las consultas que hemos visto funcionan de fila en fila, mediante una cláusula WHERE para filtrar filas. Cada fila devuelta corresponde a una fila del conjunto de datos original.

Muchas funciones de agregado se proporcionan en SQL Server. En esta sección, veremos las funciones más comunes, como SUM, MIN, MAX, AVG y COUNT.

Al trabajar con funciones de agregado, debe tener en cuenta los siguientes puntos:

- Las funciones de agregado devuelven un único valor (escalar) y se pueden usar en instrucciones SELECT casi en cualquier lugar en el que se pueda usar un solo valor. Por ejemplo, estas funciones se pueden usar en las cláusulas SELECT, HAVING y ORDER BY. Sin embargo, no se pueden usar en la cláusula WHERE.
- Las funciones de agregado omiten los valores NULL, excepto cuando se usa COUNT(*).
- Las funciones de agregado de una lista SELECT no tienen un encabezado de columna, a menos que proporcione un alias mediante AS.
- Las funciones de agregado de una lista SELECT funcionan en todas las filas que se pasan a la operación SELECT. Si no hay ninguna cláusula GROUP BY, se resumirán todas las filas que cumplan cualquier filtro de la cláusula WHERE. Obtendrá más información sobre GROUP BY en la unidad siguiente.
- A menos que use GROUP BY, no debe combinar funciones de agregado con columnas no incluidas en las funciones de la misma lista SELECT.
Para ampliar más allá de las funciones integradas, SQL Server proporciona un mecanismo para las funciones de agregado definidas por el usuario a través de Common Language Runtime (CLR) de .NET. Esta estrategia va más allá del ámbito de este módulo.

### Funciones de agregado integradas
Como se mencionó, Transact-SQL proporciona muchas funciones de agregado integradas. Estas son algunas de las más comunes:



SUM: SUM(expression) Suma todos los valores numéricos no NULL de una columna.

MEDIA: AVG(expression) Promedia todos los valores numéricos no NULL de una columna (suma/recuento).

MÍN: MIN(expression) Devuelve el número más pequeño, la fecha y hora más tempranas o la cadena que se produce por primera vez (según las reglas de ordenación de intercalación).

MÁX: MAX(expression) Devuelve el número más grande, la fecha y hora más recientes o la última cadena (según las reglas de ordenación de intercalación).

COUNT o COUNT_BIG:  COUNT(*) o COUNT(expresión) Con (*), se cuentan todas las filas, incluidas las filas con valores NULL. Cuando se especifica una columna como expresión, devuelve el recuento de filas que no son NULL para esa columna. COUNT devuelve un valor int; COUNT_BIG devuelve un valor big_int.